### Master Export

In [2]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np


# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# Open Sheet
sheet = client.open_by_key("1Feh5lJStiEkMjT0qkLl8yu81Tbi1WjIeILcxrk13FsI") # Master Sheets ( Dimensions Sheet)

#### Route Data (Coverage Data)

In [3]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
ProductMaster = sheet.worksheet("ProductMaster")
Overall_Master_Product = sheet.worksheet("Overall_Master_Product")

NPF_Master_SOS = sheet.worksheet("NPF_Master_SOS")

StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
ProductMaster = get_as_dataframe(ProductMaster)

Overall_Master_Product = get_as_dataframe(Overall_Master_Product)
NPF_Master_SOS = get_as_dataframe(NPF_Master_SOS)

StoreMasterRevised = StoreMaster[
    ['LIS Store Code', 'Old DT Store Code', 'DT Store Code', 'Amazing ID', 'Emp User ID','LIS User Id','Employee Name']
]

StoreMasterRevised = StoreMasterRevised.rename(
    columns={
        'LIS Store Code': 'New_DT_Store_Code',
        'Old DT Store Code': 'OLD_DT_Store_Code',
        'DT Store Code': 'LIS_StoreCode',
        'Amazing ID': 'Old_Amazing_EMP_ID',
        'Emp User ID': 'New_Amazing_EMP_ID',
        'LIS User Id':'LISUserId',
        'Employee Name':'EmployeeName'
    }
)

## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

StoreMasterRevised['New_Amazing_EMP_ID'] = (
    StoreMasterRevised['New_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

StoreMasterRevised['Old_Amazing_EMP_ID'] = (
    StoreMasterRevised['Old_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)



In [4]:
#### Route Data (Coverage Data)

coverage=pd.read_csv('1.Route_Data.csv',usecols = ['Check In Date','Check Out Date','Visit status','Route Day','Route Date','Store status','Merchandiser Code','Store Name','Store Code'])


coverage['Merchandiser Code'] = (coverage['Merchandiser Code'].astype(str).str.replace('@zing.com', '', regex=False))

coverage['Merchandiser Code'] = (
    coverage['Merchandiser Code']
    .astype(str)
    .str.upper()
)

#### Cleaning of camp id columm ( new amazing - consolidated)
#Creating a Consolidated EMP id or Camp Id

temp = StoreMasterRevised[['Old_Amazing_EMP_ID','New_Amazing_EMP_ID']].drop_duplicates()

coverage1 = pd.merge(coverage,temp,
                     left_on = 'Merchandiser Code',right_on='Old_Amazing_EMP_ID' , how = 'left')

coverage2 = pd.merge(coverage1,StoreMasterRevised[['New_Amazing_EMP_ID']].drop_duplicates(),left_on='Merchandiser Code',right_on='New_Amazing_EMP_ID',
                     how='left',suffixes=('_old_map', '_new_map'))

coverage2['emp_flag'] = (coverage2[['New_Amazing_EMP_ID_old_map', 'New_Amazing_EMP_ID_new_map']].notna().any(axis=1).astype(int))

coverage2['Consolidated_NEW_EMP_ID'] = (coverage2['New_Amazing_EMP_ID_old_map'].combine_first(coverage2['New_Amazing_EMP_ID_new_map']))

coverage2.drop(columns=['Old_Amazing_EMP_ID','New_Amazing_EMP_ID_old_map', 'New_Amazing_EMP_ID_new_map'],inplace=True,errors='ignore')

coverage3= coverage2[(coverage2['emp_flag']==1)]

coverage3= coverage3.rename(columns = {'Check In Date':'CheckInDate','Check Out Date':'CheckOutDate','Visit status':'VisitStatus','Merchandiser Code':'MerchandiserCode',
                                      'Store Name':'StoreName','Store Code':'StoreCode'})

## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

coverage3['Consolidated_NEW_EMP_ID'] = (
    coverage3['Consolidated_NEW_EMP_ID']
    .astype(str)
    .str.upper()
)

coverage3= coverage3.drop(columns = ['MerchandiserCode','emp_flag'])

#time

coverage3['CheckInDate'] = pd.to_datetime(coverage3['CheckInDate'], errors='coerce')
coverage3['CheckOutDate'] = pd.to_datetime(coverage3['CheckOutDate'], errors='coerce')

coverage3['CheckIn_Time_24H'] = coverage3['CheckInDate'].dt.strftime('%H:%M')
coverage3['CheckOut_Time_24H'] = coverage3['CheckOutDate'].dt.strftime('%H:%M')


time_diff = (coverage3['CheckOutDate'] - coverage3['CheckInDate']).abs()

valid_mask = time_diff.notna()

coverage3['WorkingHours'] = np.nan
coverage3.loc[valid_mask, 'WorkingHours'] = (
    (time_diff[valid_mask].dt.total_seconds() // 3600).astype(int).astype(str).str.zfill(2)
    + ':' +
    ((time_diff[valid_mask].dt.total_seconds() % 3600) // 60).astype(int).astype(str).str.zfill(2)
)

coverage3['Merchandising'] = np.where(
    (coverage3['VisitStatus'] == 'Y') &
    (coverage3['Store status'].isin(['Permission not granted', 'Store closed', '-'])),
    'N',
    coverage3['VisitStatus']
)

coverage3['Reason'] = np.where(
    (coverage3['VisitStatus'] == 'Y') & (coverage3['Merchandising'] == 'N'),
    coverage3['Store status'],
    np.nan
)

coverage4 = pd.merge(coverage3,StoreMasterRevised, left_on = ['StoreCode','Consolidated_NEW_EMP_ID'] , 
                     right_on= ['New_DT_Store_Code','New_Amazing_EMP_ID'],how = 'inner')


coverage4 = coverage4[coverage4['VisitStatus']=='Y']

final_coverage_data= coverage4[['Consolidated_NEW_EMP_ID','LISUserId','EmployeeName','StoreCode','LIS_StoreCode','StoreName','Route Date','CheckIn_Time_24H','CheckOut_Time_24H','WorkingHours','VisitStatus','Merchandising','Reason']]

final_coverage_data = final_coverage_data.drop_duplicates()
final_coverage_data.to_csv('final_coverage_data.csv')


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_32368\1908179671.py:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  coverage=pd.read_csv('1.Route_Data.csv',usecols = ['Check In Date','Check Out Date','Visit status','Route Day','Route Date','Store status','Merchandiser Code','Store Name','Store Code'])
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_32368\1908179671.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['08:52' '09:22' '09:16' ... '00:05' '01:28' '00:14']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  coverage3.loc[valid_mask, 'WorkingHours'] = (


#### OSA Raw

In [6]:
osa=pd.read_csv('OSA.csv',usecols = ['Route Date','Store Code','Store Name','NPG Code','Availability'])

osa['StockStaus'] = np.where(osa['Availability'] == 1, 1, 0)

osa_master = pd.merge(osa,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

osa_master_product = pd.merge(osa_master,ProductMaster[['Client ProductCode','ProductName','MRP','Variant','Category']], left_on = 'NPG Code', right_on= 'Client ProductCode', how = 'inner')

osa_master_product['MSL']= np.where(osa_master_product['Route Date'].isnull(), 'No','Yes')

final_osa = osa_master_product[['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','New_DT_Store_Code','Store Name','Client ProductCode',
                                'ProductName','MSL','StockStaus','MRP','Availability','Category']]

final_osa.to_csv('final_osa.csv')

#### Plano Raw

In [16]:
plano=pd.read_csv('Plano.csv', usecols = ['Category Type','Route Date','Status','Store Code','Store Name'])
# plano=pd.read_csv('Plano.csv')
plano['SliderStatus'] = np.where(plano['Status'] == 'Followed', 1, 0)
plano_master = pd.merge(plano,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

plano_master['ReasonForNonAvailability'] = np.where(plano_master['Route Date'].isnull(), 0, 1)

final_plano_master = plano_master[['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','New_DT_Store_Code','Store Name','SliderStatus','Category Type']]

final_plano_master = final_plano_master.drop_duplicates()

final_plano_master.to_csv('final_plano.csv')

#### SOS Raw Data

In [17]:
sos=pd.read_csv('SOS.csv', usecols = ['Route Date','Store Code','Store Name','NPG Code','NPG Description','Face On','End on','Stand On'])
# sos=pd.read_csv('SOS.csv')

sos_npf = pd.merge(sos,NPF_Master_SOS,left_on = 'NPG Code', right_on = 'NPG Code', how = 'left')

sos_npf['Brand2'] = np.where(sos_npf['Brand'] == 'Colgate', 'Colgate', 'Others')

sos_npf_master = pd.merge(sos_npf,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

columns_retained = ['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','Store Code','Store Name','Category','Brand2','Face On','End on','Stand On']

sos_npf_master = sos_npf_master[columns_retained]

sos_npf_master['facing'] = sos_npf_master.apply(
    lambda r: (r['Face On'] or 0)*1 + (r['End on'] or 0)*0.33 + (r['Stand On'] or 0)*0.33,
    axis=1
)

sos_npf_master= sos_npf_master.drop(columns = ['Face On','End on','Stand On'])


summary_sos = (
    sos_npf_master
    .pivot_table(
        index=[
            'Route Date',
            'New_Amazing_EMP_ID',
            'LISUserId',
            'EmployeeName',
            'Store Code',
            'Store Name',
            'Category'
        ],
        columns='Brand2',
        values='facing',
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)
# summary_df.columns.name = None

summary_sos['Total_Facings'] = summary_sos['Colgate']+summary_sos['Others']

summary_sos.drop(columns=['Others'], inplace=True)

summary_sos.to_csv('final_sos.csv')